I dette projekt vil vi gøre det muligt, at forudsige en karakter fra 1-5 ud fra tekstindholdet i en anmeldelse på Trustpilot,
og vise om sammenhængen af én virksomheds samlede vurdering passer mellem karakter og anmeldelser.
Vi har tænkt os at gøre dette ved at scrape en masse andmeldelser fra forskellige virksomheder på Truspilot ved brug af selenium. Vi vil basere vores model ud fra de indsamlede data.

Med vores model vil vi:

1. Evaluére og fastslå hvor brugbar modellen er ved at sammenligne træningsdata- og valideringsdata.
1a.Efterjustér modellens definition efter behov.



2. Forudsige hvilken karakter en anmelder vil give ud fra den anmeldselse de har skrevet. (Regression)
2a. Sammenlign med den aktuelle anmelder karakter.



3. Vis sansynligheden for predictions indenfor Trustpilots fem katagorier: Fremragende, God, Middel, Uder middel, Dårlig, for en specifik anmeldelse. (Classification)



4. Gruppere og plot ratings for én specifik virksomhed i et 3D feature space. (Clustering)



5. Brug WordCloud til og plotte de mest anvendte Negative og Positive ord.

In [1]:
#%pip install requests==2.27.1

import bs4
import csv
import requests
#import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor


%matplotlib notebook

print("OK")

print(bs4.__version__)
print(csv.__version__)
print(requests.__version__)



OK
4.8.2
1.0
2.23.0


In [2]:
allShopsRequest = []

initUrl1 = "https://dk.trustpilot.com/review/personligrevisor.dk"
initUrl2 = "https://dk.trustpilot.com/review/am.dk"

allShopsRequest.append(initUrl1)
allShopsRequest.append(initUrl2)

print("OK")


OK


In [3]:
allPagesRequest = []

def multiThreadDownload(shop):
    listofpages=""
    totalpages=None
    
    re = requests.get(shop, timeout=10)
    soup = bs4.BeautifulSoup(re.content, 'html.parser')
    
    try:
        
        totalpages = soup.find('a', attrs={'name':'pagination-button-last'}).text
        
    except:       
        listofpages = [7,6,5,4,3,2,1]
        try:
            for content in listofpages:
                totalpages = soup.find('a', attrs={'name':'pagination-button-'+str(content)}).text
                if totalpages is not None:
                    print(totalpages)
                    break
                
        except:
            print("Fejl for helvede!!!!")

    
    for i in range(int(totalpages)):
        i=i+1
        url = shop+'?page='+str(i)
        allPagesRequest.append(requests.get(url, timeout=10))


with ThreadPoolExecutor(4) as ex:
    ex.map(multiThreadDownload, allShopsRequest)




7


In [5]:
def multithreadwritecsvfile(page):
        with open('output.csv', 'a', newline='', encoding='ansi') as output_file:
            output_writer = csv.writer(output_file)
            beautistuff = bs4.BeautifulSoup(page.content,'html.parser')
            for content in beautistuff.find_all('section', attrs={'class':'styles_reviewContentwrapper__zH_9M'}):
                rating=None
                splitted=None
                title=None
                review=None

                try: 
                    rating = content.find('img',alt = True)
                    splitted = rating.get('alt').split()
                except:
                    pass

                try: 
                    title = content.find('a', attrs={'name':'review-title'}).text
                except:
                    pass
                try: 
                    review = content.find('p', attrs={'class':'typography_typography__QgicV typography_body__9UBeQ typography_color-black__5LYEn typography_weight-regular__TWEnf typography_fontstyle-normal__kHyN3'}).text
                    if len(review) > 2:
                        output_writer.writerow([splitted[2], title, review])
                except:
                    pass

with ThreadPoolExecutor(8) as ex:
    with open('output.csv', 'w', newline='', encoding='ansi') as output_file:
        output_writer = csv.writer(output_file)
        output_writer.writerow(['Rating', 'title', 'review'])
    ex.map(multithreadwritecsvfile, allPagesRequest)
    
    

LookupError: unknown encoding: ansi